In [283]:
import pandas as pd
import numpy as np
from tqdm.notebook import trange, tqdm
import jax
import optax
import matplotlib.pyplot as plt
import importlib
from pathlib import Path

# Force reload of project files
import data, train, models, metrics
importlib.reload(data)
from data import DataLoader


# Params
feature_labels = ['gage_Q','Blue','Green','Red','Nir','Swir1','Swir2'] #Must be list, even for 1 value
target_label = 'tss'
time_slice = slice('2008-01-01', None)
split_time =  np.datetime64('2017-01-01')
num_samples = 10000
sequence_length = 90
batch_size = 256
num_epochs = 50
hidden_size = 16
feature_size = len(feature_labels)
output_size = 1

data_dir = Path("../data/Sag")

dataloader = DataLoader(data_dir = data_dir,
                        basins = ['sag_daily_data','sag_daily_data_2'],
                        features = feature_labels,
                        target = target_label,
                        time_slice = time_slice,
                        split_time = split_time,
                        batch_size = batch_size, 
                        sequence_length = sequence_length,
                        train = True,
                        discharge_col = 'gage_Q',
                        zero_scale_cols = ['gage_Q','tss'],
                        calc_dt=True)

for d in dataloader:
    continue

d['xs'].shape

(176, 1)

In [311]:
df = pd.DataFrame({
    'A': [1, np.nan, np.nan, 4, np.nan, 6],
    'B': [np.nan, 2, np.nan, np.nan, 5, np.nan],
    'C': [1, np.nan, 3, np.nan, np.nan, np.nan]
})

# Subset of columns to process
columns_to_process = ['A','B','C']

# df.isnull().groupby((df.notnull().cumsum() == 0).cumsum()).sum()

# Fill in missing values for the subset of columns
# df_filled = df.copy()

for col in columns_to_process:
    # Create a mask for NaN values
    nan_mask = df[col].isna()
    
    # Calculate the number of missing measurements since a valid measurement
    df[col+"_dt"] = nan_mask.groupby((nan_mask == False).cumsum()).cumsum()

    # df[col] = df[col].ffill()

    # df[.rename(col+"_dt")]

# # Add the missing_since_last_valid information as new columns
# for col in columns_to_process:
#     df_filled[col + '_dt'] = missing_since_last_valid[col]

print("Filled DataFrame:")
print(df)
print("\ndt DataFrame:")
print(dt)

Filled DataFrame:
     A    B    C  A_dt  B_dt  C_dt
0  1.0  NaN  1.0     0     1     0
1  NaN  2.0  NaN     1     0     1
2  NaN  NaN  3.0     2     1     0
3  4.0  NaN  NaN     0     2     1
4  NaN  5.0  NaN     1     0     2
5  6.0  NaN  NaN     0     1     3

dt DataFrame:
0    0
1    1
2    0
3    1
4    2
5    3
Name: C, dtype: int64
